In [ ]:
#!/usr/bin/env python3
import json
import asyncio
import os
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# 환경 변수 로드
load_dotenv('env', override=True)

print("✅ 라이브러리 임포트 완료!")
print(f"💡 OpenAI API 키 설정: {'✅' if os.getenv('OPENAI_API_KEY') else '❌'}")


In [ ]:
class MCPMemoryAgent:
    """MCP 메모리 에이전트"""
    
    def __init__(self):
        """클라이언트 초기화"""
        self.client = None
        self.tools = None
        self.llm = None
        self.agent = None
        self.initialized = False
        
    async def initialize(self):
        """비동기 초기화"""
        if self.initialized:
            print("🔄 이미 초기화되어 있습니다.")
            return
            
        # OpenAI API 키 확인
        api_key = os.getenv('OPENAI_API_KEY')
        if not api_key:
            raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다. env 파일을 확인해주세요.")
        
        print("🔧 에이전트 초기화 중...")
        
        # ChatOpenAI 초기화
        self.llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.1)
        print("✅ LLM 초기화 완료")
        
        # MCP 클라이언트 설정
        self.client = MultiServerMCPClient({
            "memory": {
                "url": "http://localhost:8084/mcp",
                "transport": "streamable_http"
            }
        })
        print("✅ MCP 클라이언트 설정 완료")
        
        # 도구 로드
        self.tools = await self.client.get_tools()
        print(f"✅ MCP 도구 로드 완료 ({len(self.tools)}개)")
        
        # 리소스 로드
        try:
            resources = await self.client.get_resources('memory')
            if resources:
                memory_status = resources[0].as_string()
                print(f"📊 메모리 상태:\n{memory_status}")
        except Exception as e:
            print(f"⚠️ 리소스 로드 중 오류: {e}")
        
        # ReAct 에이전트 생성
        system_prompt = """
당신은 지능적인 AI 어시스턴트입니다. 지식 그래프 기반의 메모리 시스템을 활용하여 사용자와의 대화를 기억하고 관리할 수 있습니다.

### 메모리 관리 원칙:
1. **사용자 식별**: 사용자의 기본 정보(이름, 나이, 직업, 위치 등)를 파악하고 저장합니다.
2. **관계 추적**: 사람들 간의 관계, 조직과의 연결 등을 기록합니다.
3. **행동과 선호도**: 사용자의 관심사, 습관, 선호도를 관찰하고 저장합니다.
4. **목표와 계획**: 사용자의 목표, 계획, 의도를 기억합니다.

### 메모리 활용 방법:
- 대화 시작 시 기존 메모리를 검색하여 사용자 정보를 확인
- 새로운 정보 발견 시 적절한 엔티티와 관계로 저장
- 기존 정보와 연결하여 맥락 유지
- 대화가 끝나면 새롭게 알게 된 중요한 부분을 메모리에 저장

### 사용 가능한 도구들:
- create_entities: 새로운 사람, 조직, 개념 등을 생성
- create_relations: 엔티티 간의 관계 설정
- add_observations: 엔티티에 대한 새로운 정보 추가
- search_nodes: 특정 정보 검색
- read_graph: 전체 메모리 구조 확인

항상 친근하고 도움이 되는 톤으로 대화하며, 사용자의 개인정보를 안전하게 보호합니다.
"""
        
        self.agent = create_react_agent(self.llm, self.tools, prompt=system_prompt)
        print("✅ ReAct 에이전트 생성 완료")
        
        self.initialized = True
        print("\n🎉 MCP 메모리 에이전트 초기화 완료!")
        
    async def chat(self, user_input: str) -> str:
        """사용자와 채팅"""
        if not self.initialized:
            await self.initialize()
            
        try:
            # 에이전트 실행
            response = await self.agent.ainvoke({
                'messages': [HumanMessage(content=user_input)]
            })
            
            # 응답에서 AI 메시지 추출
            if response and 'messages' in response:
                ai_messages = [msg for msg in response['messages'] if hasattr(msg, 'content') and msg.type == 'ai']
                if ai_messages:
                    return ai_messages[-1].content
            
            return "응답을 생성할 수 없습니다."
            
        except Exception as e:
            error_msg = f"채팅 중 오류가 발생했습니다: {e}"
            print(f"❌ {error_msg}")
            return error_msg
    
    async def close(self):
        """클라이언트 종료"""
        if self.client:
            # MultiServerMCPClient는 close() 메서드가 없을 수 있음
            if hasattr(self.client, 'close'):
                try:
                    await self.client.close()
                except Exception as e:
                    print(f"⚠️ 클라이언트 종료 중 오류: {e}")
            print("🔴 MCP 클라이언트가 종료되었습니다.")

print("✅ MCPMemoryAgent 클래스 정의 완료!")


In [ ]:
# 글로벌 에이전트 인스턴스 생성
agent = MCPMemoryAgent()

# 초기화 실행
await agent.initialize()


In [ ]:
# 기본 인사 및 메모리 상태 확인
user_input = "안녕하세요! 현재 저장된 메모리 상태를 확인해주세요."
print(f"👤 사용자: {user_input}")

response = await agent.chat(user_input)
print(f"\n🤖 AI: {response}")


In [ ]:
# 개인 정보 저장
user_input = "안녕하세요! 저는 김철수이고 29세입니다. 삼성에서 AI 개발자로 일하고 있고, Python과 머신러닝을 좋아해요."
print(f"👤 사용자: {user_input}")

response = await agent.chat(user_input)
print(f"\n🤖 AI: {response}")


In [ ]:
# 동료 정보 및 관계 저장
user_input = "제 동료 이영희는 LG에서 데이터 사이언티스트로 일해요. 같은 대학 출신이고 현재 자연어처리 프로젝트를 함께 하고 있어요."
print(f"👤 사용자: {user_input}")

response = await agent.chat(user_input)
print(f"\n🤖 AI: {response}")


In [ ]:
# AI 관련 정보 검색
user_input = "AI와 관련된 사람들이나 정보를 검색해주세요."
print(f"👤 사용자: {user_input}")

response = await agent.chat(user_input)
print(f"\n🤖 AI: {response}")


In [ ]:
# 전체 지식 그래프 상태 확인
user_input = "지금까지 저장된 모든 정보를 정리해서 보여주세요. 전체 메모리 그래프를 확인하고 싶어요."
print(f"👤 사용자: {user_input}")

response = await agent.chat(user_input)
print(f"\n🤖 AI: {response}")


In [ ]:
# 회사 및 프로젝트 정보 저장
user_input = "제가 근무하는 삼성 AI팀은 서울 강남에 위치해 있고, 현재 ChatBot 프로젝트를 진행 중입니다. 팀장은 박민수 부장님이에요."
print(f"👤 사용자: {user_input}")

response = await agent.chat(user_input)
print(f"\n🤖 AI: {response}")


In [ ]:
# 삼성 관련 정보 검색
user_input = "삼성과 관련된 모든 정보를 찾아주세요."
print(f"👤 사용자: {user_input}")

response = await agent.chat(user_input)
print(f"\n🤖 AI: {response}")


In [ ]:
# memory.json 파일 내용 확인
try:
    with open('memory.json', 'r', encoding='utf-8') as f:
        memory_data = json.load(f)
    
    print("📊 저장된 메모리 데이터:")
    print(f"📝 엔티티 개수: {len(memory_data.get('entities', []))}")
    print(f"🔗 관계 개수: {len(memory_data.get('relations', []))}")
    
    print("\n📋 엔티티 목록:")
    for entity in memory_data.get('entities', []):
        print(f"  • {entity['name']} ({entity['entityType']})")
    
    print("\n🔗 관계 목록:")
    for relation in memory_data.get('relations', []):
        print(f"  • {relation['from_entity']} → {relation['relationType']} → {relation['to_entity']}")
        
except FileNotFoundError:
    print("❌ memory.json 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"❌ 파일 읽기 오류: {e}")


In [ ]:
# 자유 테스트용 - 원하는 질문을 입력하세요
user_input = "이영희에 대해 알고 있는 모든 정보를 알려주세요."

print(f"👤 사용자: {user_input}")
response = await agent.chat(user_input)
print(f"\n🤖 AI: {response}")


In [ ]:
# 에이전트 정리 및 종료
await agent.close()
print("\n🎉 MCP 메모리 에이전트 테스트 완료!")
print("💾 모든 대화 내용이 memory.json에 저장되었습니다.")
print("🔄 다시 테스트하려면 노트북을 재시작하고 처음부터 실행하세요.")
